In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from datetime import datetime

# Define Which Input Files to Use
The default settings will use the input files recently produced in Step 1) using the notebook `get_eia_demand_data.ipynb`. For those interested in reproducing the exact results included in the repository, you will need to point to the files containing the original `raw` EIA demand data that we querried on 10 Sept 2019.

Use the `clean_subregions` flag to clean and prepare the BAs and subregions for imputation.

In [2]:
check_reproducibility = os.getenv('CHECK_REPRODUCIBILITY', default=None)

clean_subregions = False
#clean_subregions = True



if not os.path.exists('./data'):
    os.mkdir('./data')

if check_reproducibility == 'TRUE':
    # download published files from Zenodo for comparison if not already present
    # v1.1 was used for Scientific Data paper: https://zenodo.org/record/3690240
    if not os.path.exists('./data/truggles-EIA_Cleaned_Hourly_Electricity_Demand_Data-1b31ad5'):
        !curl -L -o ./data/eia_data.zip https://zenodo.org/record/3690240/files/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data-v1.1.zip?download=1
        !unzip ./data/eia_data.zip -d ./data/
    input_path = './data/truggles-EIA_Cleaned_Hourly_Electricity_Demand_Data-1b31ad5/data/release_2019_Oct/original_eia_files'
    assert(os.path.exists(input_path)), f"Were the files accessed from Zenodo and unzipped correctly?"

# Running with step1 files
else:
    input_path = './data'
    if clean_subregions:
        input_path = './data_subregions'

# Define The Screening Algorithms

The following cell defines all of the screening algorithms used. The specific selection parameters are defined in a following cell.

In [3]:
def add_rolling_dem(df, short_hour_window):
    df["rollingDem"] = df["demand (MW)"].rolling(
        short_hour_window * 2, min_periods=1, center=True
    ).median()
    return df


def add_rolling_dem_long(df, nDays):
    df["rollingDemLong"] = df["demand (MW)"].rolling(
        nDays * 24 * 2, min_periods=1, center=True
    ).median()
    return df


def add_demand_minus_rolling_dem(df):
    diff = df['demand (MW)'] - df['rollingDem']
    df = df.assign(dem_minus_rolling=diff)
    return df


def add_demand_rel_diff_wrt_hourly(df):
    diff = df['demand (MW)'] / (df['rollingDem'] * df['hourly_median_dem_dev'])
    df = df.assign(dem_rel_diff_wrt_hourly=diff)
    diff2 = df['demand (MW)'] / (df['rollingDemLong'] * df['hourly_median_dem_dev'])
    df = df.assign(dem_rel_diff_wrt_hourly_long=diff2)
    return df


def add_delta_demand_rel_diff_wrt_hourly(df):
    diff3 = df['dem_rel_diff_wrt_hourly'].diff()
    df = df.assign(dem_rel_diff_wrt_hourly_delta_pre=diff3)
    diff4 = df['dem_rel_diff_wrt_hourly'].diff(periods=-1)
    df = df.assign(dem_rel_diff_wrt_hourly_delta_post=diff4)
    return df


# This is a global value so does not need to be added to the df
def calculate_relative_demand_difference_IQR(df):
    iqr_relative_deltas = np.nanpercentile(df['dem_rel_diff_wrt_hourly_delta_pre'], 75) - \
            np.nanpercentile(df['dem_rel_diff_wrt_hourly_delta_pre'], 25)
    return iqr_relative_deltas

    
def add_demand_minus_rolling_dem_iqr(df, irq_hours):
    rolling_window = df["dem_minus_rolling"].rolling(iqr_hours * 2, min_periods=1, center=True)
    df["dem_minus_rolling_IQR"] = rolling_window.quantile(0.75) - rolling_window.quantile(0.25)
    return df


def add_hourly_median_dem_deviations(df):
    # Create a df to hold all values to take nanmedian later
    vals_dem_minus_rolling = df['dem_minus_rolling']
    # Loop over nDays days on each side
    for i in range(-nDays, nDays+1):
        # Already initialized with zero value
        if i == 0:
            continue
        vals_dem_minus_rolling = pd.concat(
            [vals_dem_minus_rolling, df.shift(periods=i*24)['dem_minus_rolling']], axis=1)

    df['vals_dem_minus_rolling'] = vals_dem_minus_rolling.median(axis=1, skipna=True)
    # 1+vals to make it a scale factor
    return df.assign(hourly_median_dem_dev=1.+df['vals_dem_minus_rolling']/df['rollingDemLong'])


def add_deltas(df):
    diff = df['demand (MW)'].diff()
    df = df.assign(delta_pre=diff)
    diff = df['demand (MW)'].diff(periods=-1)
    df = df.assign(delta_post=diff)
    return df


def add_rolling_delta_iqr(df, iqr_hours):
    rolling_window = df["delta_pre"].rolling(iqr_hours * 2, min_periods=1, center=True)
    df["delta_rolling_IQR"] = rolling_window.quantile(0.75) - rolling_window.quantile(0.25)
    return df


def add_categories(df):
    df['category'] = np.where(df['demand (MW)'].isna(), 'MISSING', 'OKAY')
    return df


def filter_neg_and_zeros(df):
    df['category'] = np.where(df['demand (MW)'] <= 0., 'NEG_OR_ZERO', df['category'])
    filtered = np.where(df['demand (MW)'] <= 0., df['demand (MW)'], np.nan)
    df['negAndZeroFiltered'] = filtered
    df['demand (MW)'] = df['demand (MW)'].mask(df['demand (MW)'] <= 0.)
    return df

    
def filter_extrem_demand(df, multiplier):
    med = np.nanmedian(df['demand (MW)'])
    filtered = df['demand (MW)'].where(df['demand (MW)'] < med * multiplier)
    df['globalDemandFiltered'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['category'] = df['category'].mask(((df['demand (MW)'] != filtered) & \
                    (df['demand (MW)'].notna())), other='GLOBAL_DEM')
    df['demand (MW)'] = filtered
    return df


def filter_global_plus_minus_one(df):
    globalDemPlusMinusFiltered = [np.nan for _ in df.index]
    for idx in df.index:
        if df.loc[idx, 'category'] == 'GLOBAL_DEM':
            if df.loc[idx-1, 'category'] == 'OKAY':
                df.loc[idx-1, 'category'] = 'GLOBAL_DEM_PLUS_MINUS'
                globalDemPlusMinusFiltered[idx-1] = df.loc[idx-1, 'demand (MW)']
                df.loc[idx-1, 'demand (MW)'] = np.nan
            if df.loc[idx+1, 'category'] == 'OKAY':
                df.loc[idx+1, 'category'] = 'GLOBAL_DEM_PLUS_MINUS'
                globalDemPlusMinusFiltered[idx+1] = df.loc[idx+1, 'demand (MW)']
                df.loc[idx+1, 'demand (MW)'] = np.nan
    df['globalDemPlusMinusFiltered'] = globalDemPlusMinusFiltered
    return df
    

def filter_local_demand(df, multiplier_up, multiplier_down):
    
    # Filter in two steps to provide different labels for the categories
    filtered = df['demand (MW)'].where(
            (df['demand (MW)'] < df['rollingDem'] * df['hourly_median_dem_dev'] + \
                     multiplier_up * df['dem_minus_rolling_IQR']))
    df['localDemandFilteredUp'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['category'] = df['category'].mask(((df['demand (MW)'] != filtered) & \
                    (df['demand (MW)'].notna())), other='LOCAL_DEM_UP')
    df['demand (MW)'] = filtered
    
    filtered = df['demand (MW)'].where(
            (df['demand (MW)'] > df['rollingDem'] * df['hourly_median_dem_dev'] - \
                     multiplier_down * df['dem_minus_rolling_IQR']))
    df['localDemandFilteredDown'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['category'] = df['category'].mask(((df['demand (MW)'] != filtered) & \
                    (df['demand (MW)'].notna())), other='LOCAL_DEM_DOWN')
    df['demand (MW)'] = filtered
    
    return df


# Filter on a multiplier of the IQR and set
# the associated 'demand (MW)' value to NAN.
# Only consider "double deltas", hours with
# large deltas on both sides
def filter_deltas(df, multiplier):
    
    filtered = df['demand (MW)'].mask(
            ((df['delta_pre'] > df['delta_rolling_IQR'] * multiplier) & \
            (df['delta_post'] > df['delta_rolling_IQR'] * multiplier)) | \
            ((df['delta_pre'] < -1. * df['delta_rolling_IQR'] * multiplier) & \
            (df['delta_post'] < -1. * df['delta_rolling_IQR'] * multiplier)))

    df['deltaFiltered'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['category'] = df['category'].mask(((df['demand (MW)'] != filtered) & \
                    (df['demand (MW)'].notna())), other='DELTA')
    df['demand (MW)'] = filtered
    return df


# March through all hours recording previous "good"
# demand value and its index.  Calculate deltas between
# this value and next "good" hour.  If delta is LARGE
# mark NAN.
# Go forwards once, then backwards once to get all options.
def filter_single_sided_deltas(df, multiplier, rel_multiplier, iqr_relative_deltas):


    # Go through forwards first, then reverse
    prev_good_index = np.nan
    
    deltaSingleFiltered = []
    for idx in df.index:
        deltaSingleFiltered.append(np.nan)
        if np.isnan(df.loc[idx, 'demand (MW)']):
            continue
        
        
        # Initialize first good entry, this will never be flagged
        if np.isnan(prev_good_index):
            prev_good_index = idx
            
        
        # Check deltas demand and relative wrt hourly adjustment
        prev_good_delta_dem = abs(df.loc[prev_good_index, 'demand (MW)'] - df.loc[idx, 'demand (MW)'])
        prev_good_delta_dem_rel_diff_wrt_hourly = abs(df.loc[prev_good_index, 
                        'dem_rel_diff_wrt_hourly'] - df.loc[idx, 'dem_rel_diff_wrt_hourly'])

        
        # delta_rolling_IQR is over 5 days on each side so should be
        # similar regardless of which hours' we use. If delta is
        # large, mark this hour anomalous
        if (prev_good_delta_dem > df.loc[idx, 'delta_rolling_IQR'] * multiplier) and \
                (prev_good_delta_dem_rel_diff_wrt_hourly > rel_multiplier * iqr_relative_deltas):
            
            
            # If the previous "good" value was farther from expected values, then consider current hour good
            # and the previous hour will be caught on the way back through the reverse direction.
            # The max deviation from the rolling 4 day dem and the rolling 10 day dem is taken
            # to help catch cases where a large deviation pulls the rolling 4 day dem to center
            # on its values.  i.e. SCL 2016 Dec 15.
            prev_max = max(abs(1. - df.loc[prev_good_index, 'dem_rel_diff_wrt_hourly']),
                            abs(1. - df.loc[prev_good_index, 'dem_rel_diff_wrt_hourly_long']))
            current_max = max(abs(1. - df.loc[idx, 'dem_rel_diff_wrt_hourly']),
                            abs(1. - df.loc[idx, 'dem_rel_diff_wrt_hourly_long']))         
            if abs(current_max) < abs(prev_max):
                prev_good_index = idx
            
            # else, continue to filter this hour
            else:
                deltaSingleFiltered[-1] = df.loc[idx, 'demand (MW)']
                df.loc[idx, 'demand (MW)'] = np.nan
                df.loc[idx, 'category'] = 'SINGLE_DELTA'
        else:
            prev_good_index = idx

    
    df['deltaSingleFilteredFwd'] = deltaSingleFiltered
    
    
    ### Go through reversed, ~ copy of above code ###
    prev_good_index = np.nan
    
    deltaSingleFiltered = []
    for idx in reversed(df.index):
        deltaSingleFiltered.append(np.nan)
        if np.isnan(df.loc[idx, 'demand (MW)']):
            continue
        
        
        # Initialize first good entry, this will never be flagged
        if np.isnan(prev_good_index):
            prev_good_index = idx
            
        
        # Check deltas demand and relative wrt hourly adjustment
        prev_good_delta_dem = abs(df.loc[prev_good_index, 'demand (MW)'] - df.loc[idx, 'demand (MW)'])
        prev_good_delta_dem_rel_diff_wrt_hourly = abs(df.loc[prev_good_index, 
                        'dem_rel_diff_wrt_hourly'] - df.loc[idx, 'dem_rel_diff_wrt_hourly'])

        
        # delta_rolling_IQR is over 5 days on each side so should be
        # similar regardless of which hours' we use. If delta is
        # large, mark this hour anomalous
        if (prev_good_delta_dem > df.loc[idx, 'delta_rolling_IQR'] * multiplier) and \
                (prev_good_delta_dem_rel_diff_wrt_hourly > rel_multiplier * iqr_relative_deltas):
            
            
            deltaSingleFiltered[-1] = df.loc[idx, 'demand (MW)']
            df.loc[idx, 'demand (MW)'] = np.nan
            df.loc[idx, 'category'] = 'SINGLE_DELTA'
        else:
            prev_good_index = idx

    to_app = [val for val in reversed(deltaSingleFiltered)]
    df['deltaSingleFilteredBkw'] = to_app
    
    return df


def filter_runs(df):
    
    d1 = df['demand (MW)'].diff(periods=1)
    d2 = df['demand (MW)'].diff(periods=2)

    # cannot compare a dtyped [float64] array with a scalar of type [bool]
    filtered = df['demand (MW)'].mask((d1 == 0) & (d2 == 0))
    df['runFiltered'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['demand (MW)'] = filtered
    df['category'] = np.where(df['runFiltered'].notna(), 'IDENTICAL_RUN', df['category'])
    return df
    

def filter_anomalous_regions(df, width, anomalous_pct):
    
    percent_good_data_cnt = [0. for _ in df.index]
    percent_good_data_pre = [0. for _ in df.index]
    percent_good_data_post = [0. for _ in df.index]
    df['len_good_data'] = [0 for _ in df.index]
    data_quality_cnt = []
    data_quality_short = []
    start_good_data = np.nan
    end_good_data = np.nan
    for idx in df.index:
            
        # Remove the oldest item in the list
        if len(data_quality_short) > width:
            data_quality_short.pop(0)
        if len(data_quality_cnt) > 2 * width:
            data_quality_cnt.pop(0)
        
        # Add new item and don't count MISSING as 'bad' data
        if df.loc[idx, 'category'] == 'OKAY' or df.loc[idx, 'category'] == 'MISSING':
            data_quality_cnt.append(1)
            data_quality_short.append(1)
            # Track length of good data chunks
            if np.isnan(start_good_data):
                start_good_data = idx
            end_good_data = idx
        else:
            data_quality_cnt.append(0)
            data_quality_short.append(0)
            # Fill in length of good data chunk
            if not (np.isnan(start_good_data) or np.isnan(end_good_data)):
                len_good = end_good_data - start_good_data + 1
                df.loc[start_good_data:end_good_data, 'len_good_data'] = len_good
            start_good_data = np.nan
            end_good_data = np.nan

        
        # centered measurements have length 2 * width
        if len(data_quality_cnt) > 2 * width:
            percent_good_data_cnt[idx-width] = np.mean(data_quality_cnt)
        # left and right / pre and post measurements have length = width + 1
        if len(data_quality_short) > width:
            percent_good_data_pre[idx] = np.mean(data_quality_short)
            percent_good_data_post[idx-width] = np.mean(data_quality_short)



    
    anomalousRegionsFiltered = [np.nan for _ in df.index]
    for idx in df.index:
        if percent_good_data_cnt[idx] <= anomalous_pct:
            for j in range(idx-width, idx+width):
                if j < 1 or j >= len(df.index):
                    continue
                if df.loc[j, 'category'] == 'OKAY':
                    # If this is the start or end of continuous good data, don't filter
                    if percent_good_data_pre[j] == 1.0 or percent_good_data_post[j] == 1.0:
                        continue
                    if df.loc[j, 'len_good_data'] > width:
                        continue
                    df.loc[j, 'category'] = 'ANOMALOUS_REGION'
                    anomalousRegionsFiltered[j] = df.loc[j, 'demand (MW)']
                    df.loc[j, 'demand (MW)'] = np.nan
    

    df['anomalousRegionsFiltered'] = anomalousRegionsFiltered
    return df





def return_all_regions():
    return ['AEC', 'AECI', 'CPLE', 'CPLW',
    'DUK', 'FMPP', 'FPC',
    'FPL', 'GVL', 'HST', 'ISNE',
    'JEA', 'LGEE', 'MISO', 'NSB',
    'NYIS', 'PJM', 'SC', # 'OVEC',
    'SCEG', 'SEC', 'SOCO',
    'SPA', 'SWPP', 'TAL', 'TEC',
    'TVA', 'ERCO',
    'AVA', 'AZPS', 'BANC', 'BPAT',
    'CHPD', 'CISO', 'DOPD',
    'EPE', 'GCPD', 'IID',
    'IPCO', 'LDWP', 'NEVP', 'NWMT',
    'PACE', 'PACW', 'PGE', 'PNM',
    'PSCO', 'PSEI', 'SCL', 'SRP',
    'TEPC', 'TIDC', 'TPWR', 'WACM',
    'WALC', 'WAUW']

# append subregions to regions list
def append_sub_regions(l):
    subregions = [
        'CISO-PGAE', 'CISO-SCE', 'CISO-SDGE', 'CISO-VEA',

        'ERCO-COAS', 'ERCO-EAST', 'ERCO-FWES', 'ERCO-NCEN',
        'ERCO-NRTH', 'ERCO-SCEN', 'ERCO-SOUT', 'ERCO-WEST',

        'ISNE-4001', 'ISNE-4002', 'ISNE-4003', 'ISNE-4004',
        'ISNE-4005', 'ISNE-4006', 'ISNE-4007', 'ISNE-4008',

        'MISO-0001', 'MISO-0004', 'MISO-0006', 'MISO-0027', 'MISO-0035', 'MISO-8910',

        'NYIS-ZONA', 'NYIS-ZONB', 'NYIS-ZONC', 'NYIS-ZOND', 'NYIS-ZONE',
        'NYIS-ZONF', 'NYIS-ZONG', 'NYIS-ZONH', 'NYIS-ZONI', 'NYIS-ZONJ', 'NYIS-ZONK',

        'PJM-AE', 'PJM-AEP', 'PJM-AP', 'PJM-ATSI', 'PJM-BC', 'PJM-CE', 'PJM-DAY',
        'PJM-DEOK', 'PJM-DOM', 'PJM-DPL', 'PJM-DUQ', 'PJM-EKPC', 'PJM-JC',
        'PJM-ME', 'PJM-PE', 'PJM-PEP', 'PJM-PL', 'PJM-PN', 'PJM-PS', 'PJM-RECO',

        'PNM-KAFB', 'PNM-KCEC', 'PNM-LAC', 'PNM-NTUA', 'PNM-PNM', 'PNM-TSGT',

        'SWPP-CSWS', 'SWPP-EDE', 'SWPP-GRDA', 'SWPP-INDN', 'SWPP-KACY',
        'SWPP-KCPL', 'SWPP-LES', 'SWPP-MPS', 'SWPP-NPPD', 'SWPP-OKGE', 'SWPP-OPPD',
        'SWPP-SECI', 'SWPP-SPRM', 'SWPP-SPS', 'SWPP-WAUE', 'SWPP-WFEC', 'SWPP-WR',    
    ]
    for subR in subregions:
        l.append(subR)
    return l

# Remobes BAs with subregions to avoid double counting in the imputation step.
# This excludes the ERCO subregions because of their 2019 data start date.
def remove_regions(l):
    
    to_exclude = [
        # because of 2019 subregion start dates
        'ERCO-COAS', 'ERCO-EAST', 'ERCO-FWES', 'ERCO-NCEN',
        'ERCO-NRTH', 'ERCO-SCEN', 'ERCO-SOUT', 'ERCO-WEST',
        
        # because subregions are so small there are too many
        # identical run filters that result in many anomalous regions
        'PNM-KAFB', 'PNM-KCEC', 'PNM-LAC', 'PNM-NTUA', 'PNM-PNM', 'PNM-TSGT',
        
        # This PJM region has last data entry April 2019
        'PJM-RECO',
        
        # BAs with subregions
        'CISO', 'ISNE', 'MISO', 'NYIS', 'PJM', 'SWPP',
    ]
    
    for rm in to_exclude:
        if rm in l:
            l.remove(rm)
    
    return l

# Screening selection parameters
For a detailed description of each, please see the associated Data Descriptor linked in the repository's README.

In [4]:
short_hour_window = 24 # 48 hour moving median (M_{t,48hr})
iqr_hours = 24*5 # width in hours of IQR values of relative deviations from diurnal cycle template (IQR_{dem,t})
nDays = 10 # Used for normalized hourly demand template (h_{t,diurnal}) and 480 hour moving median (M_{t,480hr})
global_dem_cut = 10 # threshold selection for global demand filter
local_dem_cut_up = 3.5 # upwards threshold for local demand filter
local_dem_cut_down = 2.5 # downwards threshold for local demand filter
delta_multiplier = 2 # selection threshold for double-sided delta filter
delta_single_multiplier = 5 # selection threshold for single-sided delta filter
rel_multiplier = 15 # other selection threshold for single-sided delta filter
anomalous_regions_width = 24 # width in hours of anomalous region filter
anomalous_pct = .85 # required pct of good data in anomalous region filter

# Run the screening code over each EIA balancing authority

In [ ]:
regions = return_all_regions()

if clean_subregions:
    regions = append_sub_regions(regions)


regions.sort()
for region in regions:
    print(f'Processing {region}')


    df = pd.read_csv(f'{input_path}/{region}.csv',
                    dtype={'demand (MW)':np.float64},
                    na_values=['MISSING', 'EMPTY'])
        
    # Convert date/time
    df['date_time'] = pd.to_datetime(df['date_time'])
        
    # Add category labels to track which algo screens an hourly value
    df = add_categories(df)

    # Mark missing and empty values
    df = df.assign(missing=df['demand (MW)'].isna())




    #---------------------------------------------
    # Screening Step 1
    #---------------------------------------------
    
    # Set all negative and zero values to NAN
    # (negative or zero filter)
    df = filter_neg_and_zeros(df)

    # Set last demand values in runs of 3+ to NAN
    # (identical run filter)
    df = filter_runs(df)

    # Global demand filter on 10x the median value
    # (global demand filter)
    df = filter_extrem_demand(df, global_dem_cut)

    # Filter +/- 1 hour from any global deman filtered hours
    # (global demand plus/minus 1 hour filter)
    df = filter_global_plus_minus_one(df)



    
    #---------------------------------------------
    # Calculate demand characteristics for Step 2
    #---------------------------------------------
    
    # 48 hour moving median (M_{t,48hr})
    df = add_rolling_dem(df, short_hour_window)

    # 480 hour moving median (M_{t,480hr})
    df = add_rolling_dem_long(df, nDays)

    # demand minus moving median (Delta(d_{t},M_{t,48hr}))
    df = add_demand_minus_rolling_dem(df)

    # IQR values of relative deviations from diurnal cycle template (IQR_{dem,t})
    df = add_demand_minus_rolling_dem_iqr(df, iqr_hours)

    # demand deltas (delta(d_{t-1},d_{t}))
    df = add_deltas(df)

    # IQR values of demand deltas (IQR_{delta,t})
    df = add_rolling_delta_iqr(df, iqr_hours)

    # normalized hourly demand template (h_{t,diurnal})
    df = add_hourly_median_dem_deviations(df)

    # Demand deviation from hourly diurnal template (r_{t})
    # This adds both the short and long moving medians
    df = add_demand_rel_diff_wrt_hourly(df)

    # Hour-to-hour differences between hourly diurnal templates
    # (delta(r_{t-1},r_{t}))
    # This adds differences for both the short and long moving medians
    df = add_delta_demand_rel_diff_wrt_hourly(df)

    # Calculate the global IQR for the hour-to-hour differences 
    # between hourly diurnal templates (IQR_{r})
    # This is a global value and is not added to the dataframe
    iqr_relative_deltas = calculate_relative_demand_difference_IQR(df)


    
    #---------------------------------------------
    # Screening Step 2
    #---------------------------------------------
    
    # (local demand filter)
    df = filter_local_demand(df, local_dem_cut_up, local_dem_cut_down)
    
    # (double-sided delta filter)
    df = filter_deltas(df, delta_multiplier)
    
    # (single-sided delta filter)
    df = filter_single_sided_deltas(df, delta_single_multiplier,
                                rel_multiplier, iqr_relative_deltas)
    
    # (anomalous regions filter)
    df = filter_anomalous_regions(df, anomalous_regions_width, anomalous_pct)
    
    # Save as csv for easy viewing and pickle for computatinal ease
    print(f'Saving pickle {input_path}/pickle_{region}.pkl')
    pickle_file = open(f'{input_path}/pickle_{region}.pkl', 'wb') 
    pickle.dump(df, pickle_file)
    pickle_file.close()
    df.to_csv(f'{input_path}/csv_{region}.csv')


# Prepare a csv File for the MICE Imputation Process

In [ ]:
prep_final_output = True
print(f"prep_final_output {prep_final_output}")

regions = return_all_regions()

if clean_subregions:
    regions = append_sub_regions(regions)
    regions = remove_regions(regions)

print(f"Length of REGIONS: {len(regions)}")

regions.sort()
print(regions)
for i, region in enumerate(regions):
    if not prep_final_output:
        break
    print(f'Loading from pickle pickle_{region}.pkl')
    pickle_in = open(f'{input_path}/pickle_{region}.pkl','rb')
    if i == 0: # Load first instance to master
        master = pickle.load(pickle_in)
        cols = master.columns.tolist()
        cols.remove('date_time')
        master['date_time'] = pd.to_datetime(master['date_time'])
        master[region] = master['demand (MW)']
        master[region+'_category'] = master['category']
        master = master.drop(cols, axis=1)
        continue
        

    df = pickle.load(pickle_in)
    master[region] = df['demand (MW)']
    master[region+'_category'] = df['category']

if prep_final_output:
    print(master.head(5))
    master.to_csv(f'{input_path}/csv_MASTER.csv', index=False, na_rep='NA')
        